# Import

In [28]:
import os
import random
import glob
import re

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from tqdm import tqdm


# Fixed RandomSeed & Setting Hyperparameter

In [29]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

In [30]:
LOOKBACK, PREDICT, BATCH_SIZE, EPOCHS = 28, 7, 16, 5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data Load

In [31]:
train = pd.read_csv("./dataset/train/train.csv")

# Define Model

In [32]:
class MultiOutputLSTM(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, num_layers=2, output_dim=7):
        super(MultiOutputLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])  # (B, output_dim)

# Train

In [33]:
def train_lstm(train_df):
    trained_models = {}

    for store_menu, group in tqdm(train_df.groupby(['영업장명_메뉴명']), desc ='Training LSTM'):
        store_train = group.sort_values('영업일자').copy()
        if len(store_train) < LOOKBACK + PREDICT:
            continue

        features = ['매출수량']
        scaler = MinMaxScaler()
        store_train[features] = scaler.fit_transform(store_train[features])
        train_vals = store_train[features].values  # shape: (N, 1)

        # 시퀀스 구성
        X_train, y_train = [], []
        for i in range(len(train_vals) - LOOKBACK - PREDICT + 1):
            X_train.append(train_vals[i:i+LOOKBACK])
            y_train.append(train_vals[i+LOOKBACK:i+LOOKBACK+PREDICT, 0])

        X_train = np.array(X_train)
        y_train = np.array(y_train)

        X_train = torch.tensor(X_train).float().to(DEVICE)
        y_train = torch.tensor(y_train).float().to(DEVICE)

        model = MultiOutputLSTM(input_dim=1, output_dim=PREDICT).to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.MSELoss()

        model.train()
        for epoch in range(EPOCHS):
            idx = torch.randperm(len(X_train))
            for i in range(0, len(X_train), BATCH_SIZE):
                batch_idx = idx[i:i+BATCH_SIZE]
                X_batch, y_batch = X_train[batch_idx], y_train[batch_idx]
                output = model(X_batch)
                loss = criterion(output, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        trained_models[store_menu] = {
            'model': model.eval(),
            'scaler': scaler,
            'last_sequence': train_vals[-LOOKBACK:]  # (28, 1)
        }

    return trained_models

In [34]:
# 학습
trained_models = train_lstm(train)

Training LSTM: 100%|██████████| 193/193 [00:44<00:00,  4.34it/s]


# Prediction

In [35]:
def predict_lstm(test_df, trained_models, test_prefix: str):
    results = []

    for (store_menu,), store_test in test_df.groupby(['영업장명_메뉴명']):
        key = store_menu
        if key not in trained_models:
            continue

        model = trained_models[key]['model']
        scaler = trained_models[key]['scaler']

        store_test_sorted = store_test.sort_values('영업일자')
        recent_vals = store_test_sorted['매출수량'].values[-LOOKBACK:]
        if len(recent_vals) < LOOKBACK:
            continue

        recent_vals_df = pd.DataFrame(recent_vals, columns=['매출수량'])
        recent_vals_scaled = scaler.transform(recent_vals_df)
        x_input = torch.tensor([recent_vals_scaled]).float().to(DEVICE)

        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()

        pred_scaled_reshaped = pred_scaled.reshape(-1, 1)
        restored_pred = scaler.inverse_transform(pred_scaled_reshaped)
        restored = np.round(np.maximum(0, restored_pred)).astype(int).flatten()

        # 예측일자: TEST_00+1일 ~ TEST_00+7일
        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(PREDICT)]

        for d, val in zip(pred_dates, restored):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': store_menu,
                '매출수량': val
            })
    print(f"{store_menu} recent input:", recent_vals.flatten())
    print(f"{store_menu} scaled input:", recent_vals_scaled.flatten())
    print(f"{store_menu} prediction result:", pred_scaled)

    return pd.DataFrame(results)


In [36]:
all_preds = []

# 모든 test_*.csv 순회
test_files = sorted(glob.glob('./dataset/test/TEST_*.csv'))

for path in test_files:
    test_df = pd.read_csv(path)

    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    pred_df = predict_lstm(test_df, trained_models, test_prefix)
    all_preds.append(pred_df)
    
full_pred_df = pd.concat(all_preds, ignore_index=True)

/tmp/ipykernel_53733/1817950123.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  x_input = torch.tensor([recent_vals_scaled]).float().to(DEVICE)


화담숲카페_현미뻥스크림 recent input: [60  0  8 28 21 20 15 63  0 13 14 12 18 56 18  0  0  2  8 11 18 12  0  4
  4  7 10 41]
화담숲카페_현미뻥스크림 scaled input: [0.29126214 0.         0.03883495 0.13592233 0.10194175 0.09708738
 0.07281553 0.30582524 0.         0.0631068  0.06796117 0.05825243
 0.08737864 0.27184466 0.08737864 0.         0.         0.00970874
 0.03883495 0.05339806 0.08737864 0.05825243 0.         0.01941748
 0.01941748 0.03398058 0.04854369 0.19902913]
화담숲카페_현미뻥스크림 prediction result: [0.06731705 0.08232439 0.08818489 0.0690828  0.08302604 0.07214014
 0.08453845]
화담숲카페_현미뻥스크림 recent input: [44  0  4  2 14 20 15 38  0 40 25 33 25 21 18  0 15  8  6 16 25 19  0 25
 10 29 31 30]
화담숲카페_현미뻥스크림 scaled input: [0.21359223 0.         0.01941748 0.00970874 0.06796117 0.09708738
 0.07281553 0.18446602 0.         0.19417476 0.12135922 0.16019417
 0.12135922 0.10194175 0.08737864 0.         0.07281553 0.03883495
 0.02912621 0.0776699  0.12135922 0.09223301 0.         0.12135922
 0.04854369 0.1407767  0

# Submission

In [37]:
def convert_to_submission_format(pred_df: pd.DataFrame, sample_submission: pd.DataFrame):
    # (영업일자, 메뉴) → 매출수량 딕셔너리로 변환
    pred_dict = dict(zip(
        zip(pred_df['영업일자'], pred_df['영업장명_메뉴명']),
        pred_df['매출수량']
    ))

    final_df = sample_submission.copy()

    for row_idx in final_df.index:
        date = final_df.loc[row_idx, '영업일자']
        for col in final_df.columns[1:]:  # 메뉴명들
            final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)

    return final_df

In [38]:
sample_submission = pd.read_csv('./dataset/sample_submission.csv')
print("컬럼 목록:", full_pred_df.columns)
print(full_pred_df.head())

submission = convert_to_submission_format(full_pred_df, sample_submission)

submission.to_csv('./results/baseline_submission.csv', index=False, encoding='utf-8-sig')

컬럼 목록: Index(['영업일자', '영업장명_메뉴명', '매출수량'], dtype='object')
         영업일자            영업장명_메뉴명  매출수량
0  TEST_00+1일  느티나무 셀프BBQ_1인 수저세트     6
1  TEST_00+2일  느티나무 셀프BBQ_1인 수저세트     6
2  TEST_00+3일  느티나무 셀프BBQ_1인 수저세트     6
3  TEST_00+4일  느티나무 셀프BBQ_1인 수저세트     6
4  TEST_00+5일  느티나무 셀프BBQ_1인 수저세트     5
